In [44]:
import pandas as pd
import re
import pickle

In [14]:
from konlpy.tag import Okt

okt = Okt()

### Read Stopwords

In [20]:
stopwords = pd.read_pickle('stopwords.pkl')
stopwords

0          아
1          휴
2        아이구
3        아이쿠
4        아이고
       ...  
672       아홉
673        령
674       만큼
675    만 못하다
676        영
Length: 677, dtype: object

In [104]:
input_txt="‘국제경영 최고의 권위자 김익수 교수님과 함께 1996년 시작된  가장 오래된 경영학회’\
‘금융, 컨설팅 펌, 대기업 등 다양한 분야에서 종사하고 계신 400여명의 알럼나이’\
고려대학교 최고(最古)경영전략학회 SBC에서 43기 학회원을 모집합니다!\
SBC는 동아시아 지역전문가, 더 나아가 글로벌 리더를 목표로 합니다.\
국제 시장을 이해함에 있어서 중국 시장에 대한 이해는 필수입니다. 중국이란 거대한 시장은 다른 시장과 구별되는 이질성과 예측 불가능성을 가진 특수한 시장입니다. 알리바바, 텐센트 등 중국 거대기업은 단순히 중국 뿐 아니라 세계시장에서도 새로운 바람을 제시하고 있습니다. SBC는 동아시아 시장에 대한 면밀한 인식을 바탕으로 세계를 보는 글로벌리더 육성을 목표로 합니다."

### Data Cleansing

In [105]:
def cleantxt(txt):
    if not pd.isnull(txt):
        txt = txt.replace('\n',' ')
        txt = txt.replace('\r','')
        txt = txt.replace('(',' ')
        txt = txt.replace(')',' ')
        p = re.compile(r'[A-Za-z가-힣?.!\'\ ]+')
        txt = ''.join(re.findall(p,txt))
        txt = re.sub(' +', ' ', txt).strip()
    return txt
input_txt = cleantxt(input_txt)
input_txt

'국제경영 최고의 권위자 김익수 교수님과 함께 년 시작된 가장 오래된 경영학회금융 컨설팅 펌 대기업 등 다양한 분야에서 종사하고 계신 여명의 알럼나이고려대학교 최고 경영전략학회 SBC에서 기 학회원을 모집합니다!SBC는 동아시아 지역전문가 더 나아가 글로벌 리더를 목표로 합니다.국제 시장을 이해함에 있어서 중국 시장에 대한 이해는 필수입니다. 중국이란 거대한 시장은 다른 시장과 구별되는 이질성과 예측 불가능성을 가진 특수한 시장입니다. 알리바바 텐센트 등 중국 거대기업은 단순히 중국 뿐 아니라 세계시장에서도 새로운 바람을 제시하고 있습니다. SBC는 동아시아 시장에 대한 면밀한 인식을 바탕으로 세계를 보는 글로벌리더 육성을 목표로 합니다.'

### Tokenize

In [68]:
X = []

temp_X = []
temp_X = okt.morphs(input_txt, stem=True) # 토큰화
temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
X.append(temp_X)

### Get Trained Tokenizer

In [74]:
with open('tokenizer.pickle','rb') as f:
    tokenizer = pickle.load(f)

In [75]:
tokenizer

In [77]:
X  = tokenizer.texts_to_sequences(X)

In [78]:
from keras.preprocessing.sequence import pad_sequences
maxlen = 150

X = pad_sequences(X, padding='post', maxlen=maxlen)

### Predict Through trained model

### Load Saved LSTM Model

In [29]:
import tensorflow as tf
new_model = tf.keras.models.load_model('./model/model_3')

In [79]:
new_model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 150)]             0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 150, 100)          1204400   
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                42240     
_________________________________________________________________
dense_5 (Dense)              (None, 20)                1300      
Total params: 1,247,940
Trainable params: 1,247,940
Non-trainable params: 0
_________________________________________________________________


### Predict

In [81]:
pred = new_model.predict(X)
pred

array([[0.        , 0.        , 0.        , 0.20244864, 0.27783078,
        0.6853198 , 0.22318052, 0.26533335, 0.95503205, 0.        ,
        0.        , 0.09494039, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.08616064, 0.        ]],
      dtype=float32)

In [87]:
result=dict()
for p in pred:
    for n, _ in enumerate(p):
        result[n]=_

In [88]:
result

{0: 0.0,
 1: 0.0,
 2: 0.0,
 3: 0.20244864,
 4: 0.27783078,
 5: 0.6853198,
 6: 0.22318052,
 7: 0.26533335,
 8: 0.95503205,
 9: 0.0,
 10: 0.0,
 11: 0.094940394,
 12: 0.0,
 13: 0.0,
 14: 0.0,
 15: 0.0,
 16: 0.0,
 17: 0.0,
 18: 0.086160645,
 19: 0.0}

In [95]:
#상위4개
gettopfour = sorted(result.items(), key = lambda x: x[1], reverse=True)[:4]
gettopfour

[(8, 0.95503205), (5, 0.6853198), (4, 0.27783078), (7, 0.26533335)]

In [96]:
thres = []
for a, b in gettopfour:
    if b>0.5:
        thres.append(a)

### map with taget index

In [98]:
tagindex = {'봉사': 0,
 '예술': 1,
 '사회': 2,
 'IT': 3,
 '공모전': 4,
 '경영': 5,
 '과학': 6,
 '미술': 7,
 '대외활동': 8,
 '기타': 9,
 '운동': 10,
 '취업': 11,
 '동아리': 12,
 '언어': 13,
 '친목': 14,
 '종교': 15,
 '스터디': 16,
 '연합': 17,
 '학회': 18,
 '음악': 19}

In [102]:
final = []
for _ in thres:
    final.append(list(tagindex.keys())[_])

In [103]:
final

['대외활동', '경영']